# New Version

First, try sklearn

In [ ]:
import sklearn


In [ ]:
import re


In [ ]:
import numpy as np

In [ ]:
from sklearn import cluster

In [ ]:
LIB = sklearn

In [189]:
'AsadSLF'.lower()

'asadslf'

In [190]:


def split_func_name(func):
    new_str = ''
    for i, l in enumerate(func):
        if i>0 and l.isupper() and func[i-1].islower():
            new_str+='.'
        new_str+=l
    return re.split('\.|_', new_str.lower())

In [ ]:
with open('/projects/bdata/jupyter/decision_points/{}.txt'.format(LIB.__name__),'r') as f:
    dp  = f.read().split('\n')

In [248]:
with open('./file_funcs.json','r') as f:
    data = json.load(f)

In [ ]:
dp = [f for f in dp if f!='']

In [ ]:
vocab = []
doc_frequency = {}
for f in dp:
    tokens = split_func_name(f)
#     tokens = 
    for t in tokens:
        if t not in doc_frequency:
            doc_frequency[t] = 0
        doc_frequency[t]+=1
        if t not in vocab:
            vocab.append(t)
        

In [ ]:
len(vocab)

In [ ]:
doc_frequency

In [ ]:
word2idx = {v:i for i,v in enumerate(vocab)}

func2vector = {}
for f in dp:
    func2vector[f] = np.zeros(len(vocab))
    for t in split_func_name(f):
        func2vector[f][word2idx[t]]=1*np.log(len(dp)/doc_frequency[t])

In [ ]:
func2vector

In [ ]:
idx2func = list(func2vector.keys())
func2idx = {f: i for i, f in enumerate(idx2func)}
vectors = [func2vector[f] for f in idx2func]
occur_matrix = np.stack(vectors)
cooccur_matrix = np.dot(occur_matrix, occur_matrix.T)

In [ ]:
cooccur_matrix

In [ ]:
a = 1- cooccur_matrix/cooccur_matrix.max()

In [ ]:
a

In [ ]:
len(dp)

In [92]:
model = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=0.999, affinity="precomputed", linkage="average").fit(a)

In [93]:
model.n_clusters_

5

In [94]:
clusters = {}
for i, l in enumerate(model.labels_):
    if l not in clusters:
        clusters[l] = []
    clusters[l].append(idx2func[i])
clusters = list(clusters.values())

In [101]:
root = {}
# cur_root = root
for f in dp:
    cur_root = root
    for t in f.split('.'):
        if t not in cur_root:
            cur_root[t] = {}
        cur_root = cur_root[t]

In [102]:
print(json.dumps(root,indent=2))

{
  "sklearn": {
    "base": {
      "clone": {},
      "is_classifier": {}
    },
    "calibration": {
      "CalibratedClassifierCV": {},
      "calibration_curve": {}
    },
    "clone": {},
    "cluster": {
      "AffinityPropagation": {},
      "AgglomerativeClustering": {},
      "Birch": {},
      "DBSCAN": {},
      "FeatureAgglomeration": {},
      "KMeans": {},
      "MeanShift": {},
      "MiniBatchKMeans": {},
      "OPTICS": {},
      "SpectralBiclustering": {},
      "SpectralClustering": {},
      "SpectralCoclustering": {},
      "affinity_propagation": {},
      "bicluster": {
        "SpectralBiclustering": {},
        "SpectralCoclustering": {}
      },
      "birch": {
        "Birch": {}
      },
      "estimate_bandwidth": {},
      "hierarchical": {
        "AgglomerativeClustering": {}
      },
      "k_means": {},
      "spectral_clustering": {}
    },
    "covariance": {
      "EllipticEnvelope": {},
      "EmpiricalCovariance": {},
      "GraphLasso": {},
   

In [115]:
freq_root = []
threshold = 4
def traverse_tree(prefix, root):
    for k in root:
        cur_prefix = '{}.{}'.format(prefix, k) if prefix!='' else k
        if len(root[k])>threshold:
            freq_root.append(cur_prefix)
        traverse_tree(cur_prefix, root[k])
            

In [116]:
traverse_tree('', root)

In [119]:
freq_root = [r for r in freq_root if r!=LIB.__name__]

In [120]:
len(freq_root)

20

In [247]:
# for i, r in enumerate(freq_root):
#     print('-'*20)
r = freq_root[20]
funcs = [f[len(r)+1:] for f in dp if f.startswith(r)]


# for f in f

vocab = []
doc_frequency = {}
for f in funcs:
#     print(f)
    tokens = split_func_name(f)
#     tokens = 
    for t in tokens:
        if t not in doc_frequency:
            doc_frequency[t] = 0
        doc_frequency[t]+=1
        if t not in vocab:
            vocab.append(t)
        
word2idx = {v:i for i,v in enumerate(vocab)}


func2vector = {}
for f in funcs:
    func2vector[f] = np.zeros(len(vocab))
    for t in split_func_name(f):
        func2vector[f][word2idx[t]]=1
            
idx2func = list(func2vector.keys())
func2idx = {f: i for i, f in enumerate(idx2func)}
vectors = [func2vector[f] for f in idx2func]
occur_matrix = np.stack(vectors)
cooccur_matrix = np.dot(occur_matrix, occur_matrix.T)
a = 1- cooccur_matrix/cooccur_matrix.max()
model = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=0.8, affinity="precomputed", linkage="average").fit(a)
print(model.n_clusters_)
    
clusters = {}
for i, l in enumerate(model.labels_):
    if l not in clusters:
        clusters[l] = []
    clusters[l].append(idx2func[i])
clusters = list(clusters.values())
for c in clusters:
    print('-'*20)
#     print(c)
    for f in c:
        print('{}.{}'.format(r, f))

IndexError: list index out of range

In [319]:
# for i, r in enumerate(freq_root):
#     print('-'*20)
r = freq_root[19]
funcs = [f for f in dp if f.startswith(r)]



func2vector = {}
for f in funcs:
    func2vector[f] = np.zeros(len(data))
    
idx2file = list(data.keys())
file2idx = {f:i for i, f in enumerate(idx2file)}
    
for d in data:
    for f in set(data[d]["funcs"]):
        if f in func2vector:
            func2vector[f][file2idx[d]]=1
    
    
idx2func = list(func2vector.keys())
func2idx = {f: i for i, f in enumerate(idx2func)}
vectors = [func2vector[f] for f in idx2func]
occur_matrix = np.stack(vectors)
cooccur_matrix = np.dot(occur_matrix, occur_matrix.T)
a = 1- cooccur_matrix/cooccur_matrix.max()
model = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=1, affinity="precomputed", linkage="average").fit(a)
print(model.n_clusters_)
    
clusters = {}
for i, l in enumerate(model.labels_):
    if l not in clusters:
        clusters[l] = []
    clusters[l].append(idx2func[i])
clusters = list(clusters.values())
for c in clusters:
    print('-'*20)
    for f in c:
        print('{}.{}'.format(r, f))

5
--------------------
sklearn.utils.extmath.sklearn.utils.extmath.cartesian
--------------------
sklearn.utils.extmath.sklearn.utils.extmath.density
--------------------
sklearn.utils.extmath.sklearn.utils.extmath.randomized_svd
--------------------
sklearn.utils.extmath.sklearn.utils.extmath.softmax
--------------------
sklearn.utils.extmath.sklearn.utils.extmath.weighted_mode


In [186]:
from sklearn import *

In [184]:
import sklearn

In [144]:
func2vector

{'base.Bunch': array([4.12713439, 4.12713439, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.

In [145]:
len(vocab)

79

___

# Last Edited

In [ ]:
with open('./sklearn_dp.txt','r') as f:
    sklearn_dps = f.read().split()

In [ ]:
import json
with open('./func_counter.json','r') as f:
    func_counter = json.load(f)["func_counter"]
statsmodels_counter = {k: func_counter[k] for k in func_counter if k.startswith('sklearn.')}

In [ ]:
target_funcs = sklearn_dps

In [ ]:
statsmodels_counter

In [ ]:
sklearn_dps

In [ ]:
sklearn_roots = ['.'.join(f.split('.')[:-1]) for f in sklearn_dps]

In [ ]:
from importlib import import_module

In [ ]:
sklearn_dp  = sklearn_dps

In [ ]:
roots_counter = {}
for r in sklearn_roots:
    if r not in roots_counter:
        roots_counter[r]=0
    roots_counter[r]+=1
frequent_roots = {r: roots_counter[r] for r in roots_counter if roots_counter[r]>=5}
for r in frequent_roots:
#     if r=="statsmodels":continue
    try:
        mod = import_module(r)
#         print(mod.__all__)
        try:
            dps = ['{}.{}'.format(r, obj) for obj in mod.__all__] 
        except:
            dps =  ['{}.{}'.format(r, obj) for obj in dir(mod) if not obj.startswith('_')]
#         dps = ['{}.{}'.format(r, obj) for obj in dir(mod) if not obj.startswith('_')]
        sklearn_dp+=dps
#         print(dps)
    except Exception as e:
        print(e)
sklearn_dp = sorted(list(set(sklearn_dp)))

In [ ]:
len(sklearn_dp)

In [ ]:
funcs = [f for f in sklearn_dp if f.endswith("Regressor") or f.endswith("Classifier")]

In [ ]:
funcs

In [ ]:
target_funcs = funcs

In [ ]:
nb_path = '/home/gezhang/data/jupyter/target'
notebooks = os.listdir(nb_path)

In [ ]:
import os

In [ ]:
sklearn_notebooks = []
for i, nb in enumerate(notebooks):
    if i%10000==0:
        print(i)
    with open(os.path.join(nb_path, nb),'r') as f:
#         print('--')
        content = f.read()
    tokens = content.split()
    if 'sklearn' in tokens:
        sklearn_notebooks.append(nb)
    

In [ ]:
len(sklearn_notebooks)

In [ ]:
sklearn_notebooks = sklearn_notebooks[:20000]

In [ ]:
from extract_func import process_file

In [ ]:
func2vector = {}
vector_size = len(sklearn_notebooks)
err_files = []

In [ ]:
for f in target_funcs:
    func2vector[f] = np.zeros(vector_size)

In [ ]:
import numpy as np

In [ ]:
for i, nb in enumerate(sklearn_notebooks):
    if i%10000 == 0:
        print('Log: {} notebooks processed'.format(i))
    funcs = []
    try:
        funcs, linenos = process_file(os.path.join(nb_path, nb))
    except Exception as e:
        err_files.append(nb)
#     funcs = [func for func in funcs if func in no_loop_sklearn_funcs]
    for func in funcs:
        if func not in target_funcs:
            continue
#         if not func.startswith('sklearn'):
#             continue
#         if func not in func2vector:
#             func2vector[func] = np.zeros(vector_size)
        func2vector[func][i] = 1

In [ ]:
idx2func = list(func2vector.keys())
func2idx = {f: i for i, f in enumerate(idx2func)}
vectors = [func2vector[f] for f in idx2func]
occur_matrix = np.stack(vectors)
cooccur_matrix = np.dot(occur_matrix, occur_matrix.T)

In [ ]:
cooccur_matrix.shape

In [ ]:
a = 1- cooccur_matrix/cooccur_matrix.max()

In [ ]:
from sklearn import cluster


In [ ]:
model = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=0.4, affinity="precomputed", linkage="average").fit(a)

In [ ]:
clusters = []
n_samples = len(idx2func)
for child in model.children_:
    clusters.append([])
    for branch in child:
        if branch < n_samples:
            clusters[-1].append(idx2func[branch])
        else:
            clusters[-1]+=clusters[branch-n_samples]

# cooccurence

In [ ]:
for c in clusters:
    if len(c)<=10:
#         print(c)
        print('-'*20)
#         print(c)
        for f in c:
            print(f)


In [ ]:
split_func_name("sklearn.ensemble.AdaBoostClassifier")

In [ ]:
len(vocab)

In [ ]:
func2vector2

In [ ]:
clusters2 = []
n_samples2 = len(idx2func2)
for child in model2.children_:
    clusters2.append([])
    for branch in child:
        if branch < n_samples2:
            clusters2[-1].append(idx2func2[branch])
        else:
            clusters2[-1]+=clusters2[branch-n_samples2]

In [ ]:
clusters2

# Common Sub Token

In [ ]:
for c in clusters2:
    if len(c)<=15 and len(c)>3:
#         print(c)
        print('-'*20)
#         print(c)
        for f in c:
            print(f)

In [ ]:
occur_matrix2.shape

In [ ]:
occur_matrix.shape

In [ ]:
m = np.append(occur_matrix, occur_matrix2, axis = 1)

In [ ]:
model2 = cluster.AgglomerativeClustering(n_clusters = None, distance_threshold=0.4).fit(m)

In [ ]:
clusters3 = []
n_samples2 = len(idx2func2)
for child in model2.children_:
    clusters3.append([])
    for branch in child:
        if branch < n_samples2:
            clusters3[-1].append(idx2func2[branch])
        else:
            clusters3[-1]+=clusters3[branch-n_samples2]

In [ ]:
clusters3

# sub token+ cooccur

In [ ]:
for c in clusters3:
    if len(c)<=15 and len(c)>3:
#         print(c)
        print('-'*20)
#         print(c)
        for f in c:
            print(f)